# TP Métaheuristiques

## 1ère étape

### Lecture des jeux de données

https://homepages.laas.fr/huguet/drupal/content/métaheuristiques-4ir-info#overlay-context=content/m%25C3%25A9ta-heuristiques-4ir-info  

In [1]:
# Classes

class Evacuation:
    # Liste des noeuds à évacuer
    # Nombre de noeuds à évacuer
    # Le noeud sur
    
    def __init__(self):
        self.listeNoeudsE = []
    def sur(self, c):
        self.nombreNoeudsE = int(c[0])
        self.noeudSur = int(c[1])
    def ajoutNoeudE(self, c):
        nouveauNoeudE = NoeudAEvacuer(c)
        self.listeNoeudsE.append(nouveauNoeudE)
        
class NoeudAEvacuer:
    # ID/Population/taux d'évacuation max
    # Chemin d'évacuation, composé des id des noeuds par lesquels passer
    def __init__(self, c):
        self.nom = int(c[0])
        self.pop = int(c[1])
        self.tauxE = int(c[2])
        self.cheminE = [int(noeudId) for noeudId in c[4:]]
    def setTauxE(self, newTauxE):
        self.tauxE = newTauxE
        
class Graphe: 
    # Liste d'arcs UTILES
    # Nombre d'arcs UTILES
    def __init__(self):
        self.listeArcs = []
    def nombre(self,c):
        self.nombreArcs = int(c[1])
    def ajoutArc(self, c):
        nouveauArc = Arc(c)
        self.listeArcs.append(nouveauArc)
        # On ajoute le même arc avec ses bornes inversées.
        x = int(c[0])
        c[0] = int(c[1])
        c[1] = x
        nouveauArc = Arc(c)
        self.listeArcs.append(nouveauArc)

class Arc: 
    # Arc origine/destination(int) ??  /Durée(int)/Capa(int) 
    def __init__(self, c):
        self.origine = int(c[0])
        self.destination = int(c[1])
        self.duree = int(c[3])
        self.capa = int(c[4])
        
# ------------------------------------

# Fonction principale

def lecture(nomFichier): 
    evacuation = Evacuation()
    graphe = Graphe()
    with open("./Ressources/" + nomFichier, "r") as fichier :
        # print ("Name of the file: " + fichier.name)

        # Liste des lignes du fichier
        lignes = fichier.readlines()
        
        # Traitement de l'instance
        val = 1
        for ligne in lignes[1:]: 
            c = ligne.split()
            if c[0] == 'c': 
                val = 2
            else:
                if val == 1:
                    traitementEvacuation(c, evacuation)
                elif val == 2:
                    traitementGraphe(c, graphe)
                else :
                    sys.exit("[Erreur] lecture(nomFichier, arbre, graphe)")
                    
        # Suppression des arcs inutiles
        majArcsUtiles(evacuation, graphe)
        
        # Mise à jour du taux d'évacuation des noeuds à évacuer
        for noeudE in evacuation.listeNoeudsE: 
            origines = noeudE.cheminE[:-1]
            origines.append(noeudE.nom)
            capacites = [arc.capa for arc in graphe.listeArcs if (arc.origine in origines)]
            noeudE.setTauxE(min(capacites))
            
        # Fermeture du fichier
        fichier.close()
        # Fin de la lecture
        
        return evacuation, graphe

# ---------------------------------

# Fonctions annexes

def traitementEvacuation(c, evacuation):
    if len(c) != 2:
        evacuation.ajoutNoeudE(c)
    else:
        evacuation.sur(c)
        
def traitementGraphe(c, graphe):
    if len(c) != 2: 
        graphe.ajoutArc(c)
    else:
        graphe.nombre(c)

def majArcsUtiles(evacuation, graphe):    
    # Création d'une liste d'arcs vraiment utiles issus de l'arbre.
    arcsUtiles = []
    for noeud in evacuation.listeNoeudsE:
        arc = [noeud.nom, noeud.cheminE[0]]
        if arc not in arcsUtiles:
            arcsUtiles.append(arc)
        i = 0
        while True:
            arc = [noeud.cheminE[i], noeud.cheminE[i+1]]
            if arc not in arcsUtiles:
                arcsUtiles.append(arc)
            i = i + 1
            if(noeud.cheminE[i] == evacuation.noeudSur):
                break
        
    # Comparaison des arcs du graphe avec la liste arcsUtiles.
    nouveauxArcs = []
    for arcPossible in graphe.listeArcs:
        if [arcPossible.origine,arcPossible.destination] in arcsUtiles:
            nouveauxArcs.append(arcPossible)
            
    graphe.listeArcs = nouveauxArcs
    graphe.nombreArcs = len(graphe.listeArcs)

    
def affichage(evacuation, graphe):
    print("")
    print("=============================================")
    print("--- E V A C U A T I O N ---")
    print("Noeud sûr : %s." %(evacuation.noeudSur))
    print("Nombre de noeuds à évacuer : %s." %(evacuation.nombreNoeudsE)) 
    print("ID - Pop - TauxE (+ Chemin)")
    for noeudE in evacuation.listeNoeudsE:
        print("%s     %s     %s" %(noeudE.nom, noeudE.pop, noeudE.tauxE))
        print(*noeudE.cheminE, sep = " ==> ")
        print("------------------------------------")
    print("")
    print("--- A R B R E ---")
    print("Nombre d'arcs : %s." %(graphe.nombreArcs))
    print("Orig/Dest -- Durée -- Capacité")
    for arc in graphe.listeArcs:
        print("  %s/%s       %s      %s" %(arc.origine, arc.destination, arc.duree, arc.capa))
        print("------------------------------------")
    print("=============================================")    

Testons la méthode de lecture des données sur l'exemple du cours : 

In [2]:
def testLecture(nomFichier): 
    evacuation, graphe = lecture(nomFichier)
    affichage(evacuation, graphe)

testLecture("sparse_10_30_3_2_I.full")


--- E V A C U A T I O N ---
Noeud sûr : 114.
Nombre de noeuds à évacuer : 10.
ID - Pop - TauxE (+ Chemin)
117     4316     70
64 ==> 220 ==> 173 ==> 228 ==> 75 ==> 18 ==> 2 ==> 252 ==> 167 ==> 110 ==> 267 ==> 14 ==> 190 ==> 114
------------------------------------
199     4230     72
213 ==> 149 ==> 21 ==> 208 ==> 206 ==> 80 ==> 66 ==> 61 ==> 75 ==> 18 ==> 2 ==> 252 ==> 167 ==> 110 ==> 267 ==> 14 ==> 190 ==> 114
------------------------------------
271     3514     74
8 ==> 163 ==> 144 ==> 129 ==> 24 ==> 224 ==> 74 ==> 16 ==> 232 ==> 32 ==> 114
------------------------------------
84     129     71
5 ==> 98 ==> 236 ==> 93 ==> 105 ==> 204 ==> 17 ==> 81 ==> 13 ==> 97 ==> 77 ==> 148 ==> 54 ==> 48 ==> 90 ==> 227 ==> 189 ==> 86 ==> 235 ==> 225 ==> 10 ==> 146 ==> 63 ==> 246 ==> 39 ==> 44 ==> 232 ==> 32 ==> 114
------------------------------------
24     4541     134
224 ==> 74 ==> 16 ==> 232 ==> 32 ==> 114
------------------------------------
164     1152     74
248 ==> 261 ==> 183 ==> 74 =

### Vérification et évaluation d’une solution

#### Données de l'exemple

cf exemple_cours.txt

#### Fichier solution de l'exemple

cf exemple_cours_solution.txt

#### Vérification de la solution

* Vérifier si la solution est réalisable = Respecte les contraintes.
    * vérifier à chaque instant t si la capacité de l'arc n'est pas dépassée = simulation
* Vérifier la valeur de la fonction objectif. Pour chaque noeud : 
    * trouver le nombre de personnes à évacuer
    * calculer la durée de transit des personnes de ce noeud : transit = noeudE.pop / solution.tauxE
    * calculer la durée du trajet du noeud à évacuer vers le noeud sécurisé : trajet = somme des durées des arcs du chemin d'évacuation
    * calculer la durée dévacuation de ce noeud : duree = transit + trajet

La valeur de la fonction objectif est le max des durées trouvées pour chaque noeud à évacuer. 

In [3]:
import numpy as np
import pandas as pd

# Fonction principale
    
def verification(evacuation, graphe, solution):
    result = True
    valFctObjectif = valFonctionObjectif(evacuation, graphe, solution)
    if(valFctObjectif != solution.valFctObjectif):
        result = False
        print("\nValeur de la fonction objectif erronée : " + str(valFctObjectif) + " au lieu de " + str(solution.valFctObjectif) +".")
    else:   
    # Vérifier si la solution est réalisable
        # print("\n[DEBUG] Valeur de la fonction objectif : " + str(result))
        # Génération du diagramme de Gantt  
        gantt = simulation(evacuation, graphe, solution, valFctObjectif)
        # On vérifie que la capacité de chaque arc est bien respectée
        # affichage_gantt(gantt)
        simu = True
        for arc in graphe.listeArcs:
            if(gantt.loc[arc.origine, :] > arc.capa).any():
                simu = False
                result = False
        # print("[DEBUG] Simulation = " + str(simu))
    
    return result, gantt
        
# ---------------------------------

# Fonctions annexes

def valFonctionObjectif(evacuation, graphe, solution) : 
    # Vérifier la valeur de la fonction objectif
    # Pour chaque noeud à évacuer : 
    durees = []

    for noeudE in evacuation.listeNoeudsE: 
        # Trouver le nombre de personnes à évacuer 
        # Calculer la durée de transit des personnes de ce noeud : 
            # transit = noeudE.pop / solution.tauxE
        bonNoeudE = [noeudESol for noeudESol in solution.listeNoeudsESol if (noeudE.nom == noeudESol.nom)]
        transit = round(noeudE.pop / bonNoeudE[0].tauxE)
        # Calculer la durée du trajet du noeud à évacuer vers le noeud sécurisé : 
            # trajet = somme des durées des arcs du chemin d'évacuation
        origines = noeudE.cheminE[:-1]
        origines.append(noeudE.nom)
        trajet = round(sum([arc.duree for arc in graphe.listeArcs if (arc.origine in origines)]))
        
        # Calculer la durée dévacuation de ce noeud : duree = transit + trajet
        durees.append(transit + trajet + bonNoeudE[0].dateE)
    
    valFctObjectif = max(durees)
    return valFctObjectif

def lectureSolution(nomFichierSolution): 
    solution = Solution()
    with open("./Ressources/" + nomFichierSolution, "r") as fichier :
        # print ("Name of the file: " + fichier.name)

        # Liste des lignes du fichier
        lignes = fichier.readlines()
        
        # Traitement de l'instance
        solution.nom = lignes[0].split()[0]
        nbNoeudsE = int(lignes[1].split()[0])
        for ligne in lignes[2:(2+nbNoeudsE)]:
            c = ligne.split()
            solution.ajoutNoeudESol(int(c[0]), int(c[1]), int(c[2]))
        solution.nature = lignes[2+nbNoeudsE].split()[0]
        solution.valFctObjectif = int(lignes[2+nbNoeudsE+1].split()[0])
        solution.tpsCalcul = int(lignes[2+nbNoeudsE+2].split()[0])
        solution.methode = " ".join(lignes[2+nbNoeudsE+3].split())
        solution.commentaire = " ".join(lignes[2+nbNoeudsE+4].split())
                                    
        # Fermeture du fichier
        fichier.close()

    return solution

def affichageSolution(solution): 
    print("--- S O L U T I O N ---")
    print("Nom de l'instance résolue : " + solution.nom)
    print("Noeuds à évacuer : ")
    for noeud in solution.listeNoeudsESol:
        print("     ID : " + str(noeud.nom) + " - Taux d'évacuation : " + str(noeud.tauxE) + " - Date de début d'évacuation : " + str(noeud.dateE))
    print("Nature : " + solution.nature)
    print("Valeur de la fonction objectif : " + str(solution.valFctObjectif))
    print("Temps de calcul : " + str(solution.tpsCalcul))
    print("Méthode de résolution : " + solution.methode)
    print("Commentaire : " + solution.commentaire)
    print("")

def simulation(evacuation, graphe, solution, valFctObjectif): 
    # On itialise 
    gantt = np.zeros((len(graphe.listeArcs), valFctObjectif-[arc.duree for arc in graphe.listeArcs if (arc.destination == evacuation.noeudSur)][0]), dtype=int)
    gantt = pd.DataFrame(gantt, index = [arc.origine for arc in graphe.listeArcs])

    # Pour chaque noeud à évacuer, on va remplir la capacité que l'on utilise sur chaque arc à chaque instant
    for noeudE in evacuation.listeNoeudsE: 
        bonNoeudE = [noeudESol for noeudESol in solution.listeNoeudsESol if (noeudE.nom == noeudESol.nom)][0]
        transit = round(noeudE.pop / bonNoeudE.tauxE)
        # On retrouve la durée de début d'évacuation
        time = [noeud.dateE for noeud in solution.listeNoeudsESol if (noeudE.nom == noeud.nom)][0]
        origines = noeudE.cheminE[:-1]
        origines.append(noeudE.nom)
        for arc in [arc for arc in graphe.listeArcs if (arc.origine in origines)]: 
            gantt.loc[arc.origine, time:(time+transit-1)] += bonNoeudE.tauxE
            time += arc.duree

    return gantt

def affichageGantt(gantt):
    print("\n--- G A N T T ---")
    print(gantt)
    print("\n")

# ---------------------------------

# Classes

class NoeudAEvacuerSol: 
    # Identifiant
    # Taux d'évacuation
    # Date de début d'évacuation
    def __init__(self, pNom, pTauxE, pDateE): 
        self.nom = pNom
        self.tauxE = pTauxE
        self.dateE = pDateE

class Solution:
    # Nom de l'instance résolue
    # Pour chaque sommet à évacuer
        # Identifiant
        # Taux d'évacuation
        # Date de début d'évacuation
    # Nature de la solution (valid ou invalid)
    # Valeur de la fonction objectif
    # Temps de calcul
    # Méthode utilisée
    # Commentaire
    
    def __init__(self):
        self.listeNoeudsESol = []
    def ajoutNoeudESol(self, pNom, pTauxE, pDateE):
        nouveauNoeudESol = NoeudAEvacuerSol(pNom, pTauxE, pDateE)
        self.listeNoeudsESol.append(nouveauNoeudESol)

Testons la méthode de vérification d'une solution sur l'exemple du cours : 

In [4]:
def testVerification1(nomFichier, nomFichierSolution):
    evacuation, graphe = lecture(nomFichier)
    solution = lectureSolution(nomFichierSolution)
    result, gantt = verification(evacuation, graphe, solution)
    affichageSolution(solution)
    print("Solution vérifiée : " + str(result))

testVerification1("exemple_cours.txt", "exemple_cours_solution.txt")

def testVerification2(nomFichier, nomFichierSolution):
    evacuation, graphe = lecture(nomFichier)
    solution = lectureSolution(nomFichierSolution)
    result, gantt = verification(evacuation, graphe, solution)
    affichageSolution(solution)
    print("Solution vérifiée : " + str(result))

testVerification2("exemple_cours.txt", "exemple_cours_solution_prof.txt")

--- S O L U T I O N ---
Nom de l'instance résolue : exemple_cours.txt
Noeuds à évacuer : 
     ID : 1 - Taux d'évacuation : 7 - Date de début d'évacuation : 1
     ID : 2 - Taux d'évacuation : 3 - Date de début d'évacuation : 0
     ID : 3 - Taux d'évacuation : 3 - Date de début d'évacuation : 0
Nature : valid
Valeur de la fonction objectif : 36
Temps de calcul : 1000
Méthode de résolution : resolu a la main
Commentaire : *** Laure Feuillet & Mael Plantec ***

Solution vérifiée : True
--- S O L U T I O N ---
Nom de l'instance résolue : exemple_cours.txt
Noeuds à évacuer : 
     ID : 1 - Taux d'évacuation : 8 - Date de début d'évacuation : 3
     ID : 2 - Taux d'évacuation : 5 - Date de début d'évacuation : 0
     ID : 3 - Taux d'évacuation : 3 - Date de début d'évacuation : 0
Nature : valid
Valeur de la fonction objectif : 37
Temps de calcul : 1000
Méthode de résolution : resolu a la main
Commentaire : Olivier - on connait une meilleure solution

Solution vérifiée : True


## 2ème étape
### Calcul d'une borne inférieure

On prends ici le temps d'évacuation maximum parmis les noeuds à évacuer. En effet, le noeud qui seul, serait le plus long à finir, donne une borne inférieure car ajouter des noeuds de temps d'évac inférieurs ne va pas accélérer l'évacuation de ce premier. 

In [5]:
from pathlib import Path

def borneInf(nomFichier, evacuation, graphe):
    borneInfSolution = Solution()
    # Nom de l'instance résolue
    borneInfSolution.nom = nomFichier
    borne = 0
    # Noeuds évacués 
    for noeudE in evacuation.listeNoeudsE: 
        borneInfSolution.ajoutNoeudESol(noeudE.nom, noeudE.tauxE, 0)
        # On met à jour la valeur de durée
            # Calculer la durée de transit des personnes de ce noeud : 
        transit = round(noeudE.pop / noeudE.tauxE)
            # Calculer la durée du trajet du noeud à évacuer vers le noeud sécurisé : 
            # trajet = somme des durées des arcs du chemin d'évacuation
        origines = noeudE.cheminE[:-1]
        origines.append(noeudE.nom)
        trajet = round(sum([arc.duree for arc in graphe.listeArcs if (arc.origine in origines)]))
            # Calculer la durée dévacuation de ce noeud : duree = transit + trajet
        nouveauTemps = transit + trajet
        if borne < nouveauTemps :
            borne = nouveauTemps
            
    # Nature de la solution
    borneInfSolution.nature = "unknown"
        
    # Valeur de la fonction objectif
    borneInfSolution.valFctObjectif = borne

    # Temps de calcul
    borneInfSolution.tpsCalcul = 9999

    # Méthode 
    borneInfSolution.methode = "Borne inférieure"

    # Commentaire
    borneInfSolution.commentaire = "Laure Feuillet & Mael Plantec"
    
    result, gantt = verification(evacuation, graphe, borneInfSolution)
    
    if result: 
        borneInfSolution.nature = "valid"
    else: 
        borneInfSolution.nature = "invalid"
        
    return borneInfSolution
      
# ---------------------------------

# Fonctions annexes

def ecritureSolution(nomFichierSolution, solution):
    with open("./Ressources/" + nomFichierSolution, "w") as fichier :
        # Nom de l'instance résolue
        fichier.write(solution.nom + "\n")
        # Nombre de sommets à évacuer
        fichier.write(str(len(solution.listeNoeudsESol)) + "\n")
        
        for noeudESol in solution.listeNoeudsESol:
            # identifiant
            fichier.write(str(noeudESol.nom) + " ")
            # taux d'évacuation
            fichier.write(str(noeudESol.tauxE) + " ")
            # date de début d'évacuation
            fichier.write(str(noeudESol.dateE)+ "\n")

        # Nature de la solution
        fichier.write(solution.nature + "\n")
        
        # Valeur de la fonction objectif
        fichier.write(str(solution.valFctObjectif) + "\n")
            
        # Temps de calcul
        fichier.write(str(solution.tpsCalcul) + "\n")
        
        # Méthode 
        fichier.write(solution.methode + "\n")
        
        # Commentaire
        fichier.write(solution.commentaire)
        
        # Fermeture du fichier
        fichier.close()

In [6]:
def testBorneInf(nomFichier): 
    evacuation, graphe = lecture(nomFichier)
    borneInfSolution = borneInf(nomFichier, evacuation, graphe)
    # nomFichierBorneInf = Path("./Ressources/"+ nomFichier).stem + "_borneInf" + Path("./Ressources/"+ nomFichier).suffix
    affichageSolution(borneInfSolution)
    verif, gantt = verification(evacuation, graphe, borneInfSolution)
    print("Vérification Borne Inf : " + str(verif))
    
testBorneInf("sparse_10_30_3_2_I.full")

--- S O L U T I O N ---
Nom de l'instance résolue : sparse_10_30_3_2_I.full
Noeuds à évacuer : 
     ID : 117 - Taux d'évacuation : 70 - Date de début d'évacuation : 0
     ID : 199 - Taux d'évacuation : 72 - Date de début d'évacuation : 0
     ID : 271 - Taux d'évacuation : 74 - Date de début d'évacuation : 0
     ID : 84 - Taux d'évacuation : 71 - Date de début d'évacuation : 0
     ID : 24 - Taux d'évacuation : 134 - Date de début d'évacuation : 0
     ID : 164 - Taux d'évacuation : 74 - Date de début d'évacuation : 0
     ID : 163 - Taux d'évacuation : 74 - Date de début d'évacuation : 0
     ID : 236 - Taux d'évacuation : 71 - Date de début d'évacuation : 0
     ID : 124 - Taux d'évacuation : 72 - Date de début d'évacuation : 0
     ID : 228 - Taux d'évacuation : 72 - Date de début d'évacuation : 0
Nature : invalid
Valeur de la fonction objectif : 108
Temps de calcul : 9999
Méthode de résolution : Borne inférieure
Commentaire : Laure Feuillet & Mael Plantec

Vérification Borne Inf

### Calcul d'une borne supérieure

Somme de la durée d'évacuation de chaque noeud au taux d'évacuation maximum.

In [7]:
def borneSup(nomFichier, evacuation, graphe):
    
    borneSupSolution = Solution()
        
    # Nom de l'instance résolue
    borneSupSolution.nom = nomFichier
    
    # Pour chaque sommet à évacuer
    duree = 0
    for noeudE in evacuation.listeNoeudsE: 
        # identifiant & taux d'évacuation & date de début d'évacuation
        borneSupSolution.ajoutNoeudESol(noeudE.nom, noeudE.tauxE, duree)

        # On met à jour la valeur de durée
            # Calculer la durée de transit des personnes de ce noeud : 
        transit = round(noeudE.pop / noeudE.tauxE)
            # Calculer la durée du trajet du noeud à évacuer vers le noeud sécurisé : 
            # trajet = somme des durées des arcs du chemin d'évacuation
        origines = noeudE.cheminE[:-1]
        origines.append(noeudE.nom)
        trajet = round(sum([arc.duree for arc in graphe.listeArcs if (arc.origine in origines)]))
            # Calculer la durée dévacuation de ce noeud : duree = transit + trajet
        duree += transit + trajet

    # Nature de la solution
    borneSupSolution.nature = "valid"
    # Valeur de la fonction objectif
    borneSupSolution.valFctObjectif = duree
    # Temps de calcul
    borneSupSolution.tpsCalcul = 9999
    # Méthode 
    borneSupSolution.methode = "Borne superieure"
    # Commentaire
    borneSupSolution.commentaire = "*** Laure Feuillet & Mael Plantec ***"
    
    return borneSupSolution

In [8]:
def testBorneSup(nomFichier): 
    evacuation, graphe = lecture(nomFichier)
    borneSupSolution = borneSup(nomFichier, evacuation, graphe)
    # nomFichierBorneSup = Path("./Ressources/"+ nomFichier).stem + "_borneSup" + Path("./Ressources/"+ nomFichier).suffix
    affichageSolution(borneSupSolution)
    verif, gantt = verification(evacuation, graphe, borneSupSolution)
    print("Vérification Borne Sup : " + str(verif))
    
testBorneSup("sparse_10_30_3_2_I.full")

--- S O L U T I O N ---
Nom de l'instance résolue : sparse_10_30_3_2_I.full
Noeuds à évacuer : 
     ID : 117 - Taux d'évacuation : 70 - Date de début d'évacuation : 0
     ID : 199 - Taux d'évacuation : 72 - Date de début d'évacuation : 87
     ID : 271 - Taux d'évacuation : 74 - Date de début d'évacuation : 190
     ID : 84 - Taux d'évacuation : 71 - Date de début d'évacuation : 271
     ID : 24 - Taux d'évacuation : 134 - Date de début d'évacuation : 329
     ID : 164 - Taux d'évacuation : 74 - Date de début d'évacuation : 388
     ID : 163 - Taux d'évacuation : 74 - Date de début d'évacuation : 436
     ID : 236 - Taux d'évacuation : 71 - Date de début d'évacuation : 484
     ID : 124 - Taux d'évacuation : 72 - Date de début d'évacuation : 592
     ID : 228 - Taux d'évacuation : 72 - Date de début d'évacuation : 686
Nature : valid
Valeur de la fonction objectif : 748
Temps de calcul : 9999
Méthode de résolution : Borne superieure
Commentaire : *** Laure Feuillet & Mael Plantec ***


## 3ème étape
### Solutions initiales

In [9]:
# Retourne la solution initiale (borne sup) de l'instance du problème passé en paramètre.
def solutionInitiale_borneSup (nomFichier, evacuation, graphe) :
    borneSupSolution = borneSup(nomFichier, evacuation, graphe)
    return borneSupSolution

### Fonctions objectifs

In [10]:
# Retourne la valeur de la fonction objectif (temps d'évacuation) de la solution
def fonctionObjectif_tempsEvacuation (solution) : 
    return solution.valFctObjectif

### Critères d'arrêt

In [11]:
# Retourne true si on doit arrêter l'algo, false sinon (20 itérations)
def critereArret_20iterations (nbIterations):
    return (nbIterations >= 20)

### Méthodes de voisinage

In [12]:
import copy

# Retourne une liste de solutions
def voisinage_uniqueDecrementation (solution, evacuation, graphe) : 
    voisinage = []
    for noeudE in solution.listeNoeudsESol : 
        # On décrémente de 1 les dates d'évacuation non nulles
        if (noeudE.dateE != 0) : 
            voisin = copy.deepcopy(solution)
            noeudEVoisin = [noeudEVoisin for noeudEVoisin in voisin.listeNoeudsESol if (noeudE.nom == noeudEVoisin.nom)][0] 
            # Mise à jour du voisin
            # # Date de début d'évacuation
            noeudEVoisin.dateE = noeudEVoisin.dateE-1
            # # Valeur de la fonction objectif du voisin
            voisin.valFctObjectif = valFonctionObjectif(evacuation, graphe, voisin)
            # # Nature
            result = verification(evacuation, graphe, voisin)
            if result :
                voisin.nature = "valid"
            else : 
                voisin.nature = "invalid"
                
            voisinage.append(voisin)
    return voisinage

In [14]:
def testVoisinage(methodeVoisinage, nomFichier) :
    evacuation, graphe = lecture(nomFichier)
    solution = solutionInitiale_borneSup (nomFichier, evacuation, graphe)
    voisinage = methodeVoisinage(solution, evacuation, graphe)
    print("\n\t------ V O I S I N A G E ------")
    for voisin in voisinage : 
        affichageSolution(voisin)
testVoisinage(voisinage_uniqueDecrementation, "exemple_cours.txt")


	------ V O I S I N A G E ------
--- S O L U T I O N ---
Nom de l'instance résolue : exemple_cours.txt
Noeuds à évacuer : 
     ID : 1 - Taux d'évacuation : 8 - Date de début d'évacuation : 0
     ID : 2 - Taux d'évacuation : 5 - Date de début d'évacuation : 33
     ID : 3 - Taux d'évacuation : 3 - Date de début d'évacuation : 65
Nature : valid
Valeur de la fonction objectif : 94
Temps de calcul : 9999
Méthode de résolution : Borne superieure
Commentaire : *** Laure Feuillet & Mael Plantec ***

--- S O L U T I O N ---
Nom de l'instance résolue : exemple_cours.txt
Noeuds à évacuer : 
     ID : 1 - Taux d'évacuation : 8 - Date de début d'évacuation : 0
     ID : 2 - Taux d'évacuation : 5 - Date de début d'évacuation : 34
     ID : 3 - Taux d'évacuation : 3 - Date de début d'évacuation : 64
Nature : valid
Valeur de la fonction objectif : 93
Temps de calcul : 9999
Méthode de résolution : Borne superieure
Commentaire : *** Laure Feuillet & Mael Plantec ***



 ### Algorithme de recherche locale

In [15]:
def algo (nomFichier, fctSolutionInitiale, fctFonctionObjectif, fctCritereArret, fctVoisinage) : 
    nbIteration = 0
    voisinage = []
    evacuation, graphe = lecture(nomFichier)
    solutionInitiale = fctSolutionInitiale(nomFichier, evacuation, graphe)
    voisinage.append(solutionInitiale)
    meilleureSolution = solutionInitiale
        
    while(not(fctCritereArret(nbIteration))) : 
        nbIteration += 1
        # MeilleurVoisin : Voisin avec la plus petit valeur de fonction objectif
        meilleurVoisin = [voisin for voisin in voisinage if (fctFonctionObjectif(voisin) == min([fctFonctionObjectif(voisinBis) for voisinBis in voisinage]))][0]
        voisinage = fctVoisinage(meilleurVoisin, evacuation, graphe)
        # On met à jour meilleureSolution
        # print("[DEBUG] Voisins")
        for voisin in voisinage : 
            voisin.methode = "\n\t" + fctSolutionInitiale.__name__ + "\n\t" + fctFonctionObjectif.__name__ + "\n\t" + fctCritereArret.__name__ + "\n\t" + fctVoisinage.__name__
            # affichage_solution(voisin)
            if ((fctFonctionObjectif(voisin) < fctFonctionObjectif(meilleureSolution)) & (voisin.nature == "valid")) : 
                meilleureSolution = voisin
                print("Mise à jour de la meilleure S. Itération n°" + str(nbIteration))
    print("\n\n--- S O L U T I O N   F I N A L E ---")
    affichageSolution(meilleureSolution)
    
    # ATTENTION : mauvaises bornes sup et inf.
    print("--- C O M P A R A I S O N S ---")
    borneSupSolution = borneSup(nomFichier, evacuation, graphe)
    print("ValObj de la borne sup :" + str(borneSupSolution.valFctObjectif))
    borneInfSolution = borneInf(nomFichier, evacuation, graphe)
    print("ValObj de la borne inf :" + str(borneInfSolution.valFctObjectif))    
    
    return meilleureSolution

In [17]:
def test_algo(nomFichier, fctSolutionInitiale, fctFonctionObjectif, fctCritereArret, fctVoisinage) :
    solution = algo(nomFichier, fctSolutionInitiale, fctFonctionObjectif, fctCritereArret, fctVoisinage)

test_algo("exemple_cours.txt", solutionInitiale_borneSup, fonctionObjectif_tempsEvacuation, critereArret_20iterations,voisinage_uniqueDecrementation, )

Mise à jour de la meilleure S. Itération n°1
Mise à jour de la meilleure S. Itération n°2
Mise à jour de la meilleure S. Itération n°3
Mise à jour de la meilleure S. Itération n°4
Mise à jour de la meilleure S. Itération n°5
Mise à jour de la meilleure S. Itération n°6
Mise à jour de la meilleure S. Itération n°7
Mise à jour de la meilleure S. Itération n°8
Mise à jour de la meilleure S. Itération n°9
Mise à jour de la meilleure S. Itération n°10
Mise à jour de la meilleure S. Itération n°11
Mise à jour de la meilleure S. Itération n°12
Mise à jour de la meilleure S. Itération n°13
Mise à jour de la meilleure S. Itération n°14
Mise à jour de la meilleure S. Itération n°15
Mise à jour de la meilleure S. Itération n°16
Mise à jour de la meilleure S. Itération n°17
Mise à jour de la meilleure S. Itération n°18
Mise à jour de la meilleure S. Itération n°19
Mise à jour de la meilleure S. Itération n°20


--- S O L U T I O N   F I N A L E ---
--- S O L U T I O N ---
Nom de l'instance résolue